# Patchy particle STMV potential 

Implempentation of the Satelitte Tobacco Mosaic virus potential based on a coarse-grained partchy particle model.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import msmrd2
import msmrd2.tools.quaternions as quats
import msmrd2.visualization as msmrdvis
from msmrd2.potentials import patchyParticleSTMV
from msmrd2.integrators import overdampedLangevin as odLangevin

In [ ]:
# Define arbitrarily large list of patchy particles
np.random.seed(seed=1) # seed 1 good used for pentameric ring formation 
numparticles = 2
boxsize = 10
D = 1.0
Drot = 1.0
pyPartlist = [] 
positionList = [np.array([-4.0,0,0]), np.array([0.,0.,0.])]
orientation = np.array([1.,0.,0.,0.])

for i in range(numparticles):
    part = msmrd2.particle(D, Drot, positionList[i], orientation)
    pyPartlist.append(part)

In [ ]:
# Create list of particles that can be read from msmrd
# note the particles in this list will be independent from the python list.
partlist = msmrd2.integrators.particleList(pyPartlist)

In [ ]:
# Over-damped Langevin integrator definition
dt = 0.00001 #0.000005
seed = 2 # seed 2 good used for pentameric ring formation; seed = -1 uses random device as seed
bodytype = 'rigidbody' # three rotational degrees of freedom
integrator = odLangevin(dt, seed, bodytype) 
integrator.setKbT(0.1)

In [ ]:
# Define boundary (choose either spherical or box)
boxBoundary = msmrd2.box(boxsize,boxsize,boxsize, 'periodic')
integrator.setBoundary(boxBoundary)

In [ ]:
# Define Patchy Particle potential
strength = 100.0
angularStrength = 100.0
patchySTMV = patchyParticleSTMV(strength, angularStrength)
integrator.setPairPotential(patchySTMV)

In [ ]:
# Integrate the particles, save to .xyz to produce VMD output (additional overhead)
timesteps = 500000
stride = 2500 #1000
datafile  = open('../../data/vmd/patchyParticleSTMV.xyz', 'w')
for i in range(timesteps):
    if i%stride == 0:
        datafile.write(str(8*len(partlist)) + '\n')
        datafile.write(str(0) + '\n')
    for j, part in enumerate(partlist):
        if i%stride == 0:
            m1 = patchySTMV.getPartPosition('m1', part)
            m2 = patchySTMV.getPartPosition('m2', part)
            m3 = patchySTMV.getPartPosition('m3', part) 
            i1 = patchySTMV.getPartPosition('i1', part) 
            i2 = patchySTMV.getPartPosition('i2', part) 
            i3 = patchySTMV.getPartPosition('i3', part) 
            i4 = patchySTMV.getPartPosition('i4', part) 
            i5 = patchySTMV.getPartPosition('i5', part) 
            datafile.write('type_0' + ' ' + ' '.join(map(str, m1)) + '\n')
            datafile.write('type_1' + ' ' + ' '.join(map(str, m2)) + '\n')
            datafile.write('type_2' + ' ' + ' '.join(map(str, m3)) + '\n')
            datafile.write('type_3' + ' ' + ' '.join(map(str, i1)) + '\n')
            datafile.write('type_3' + ' ' + ' '.join(map(str, i2)) + '\n')
            datafile.write('type_3' + ' ' + ' '.join(map(str, i3)) + '\n')
            datafile.write('type_3' + ' ' + ' '.join(map(str, i4)) + '\n')
            datafile.write('type_3' + ' ' + ' '.join(map(str, i5)) + '\n')
    integrator.integrate(partlist)
    if i%5000 == 0:
        print("Percentage complete: ", 100*i/timesteps, "%", end="\r")
datafile.close()
# Generate TCL script to visualize with VMD
msmrdvis.generateTCL_patchyParticleSTMV(numparticles = numparticles, 
                                     outfname = "patchyParticleSTMV", 
                                     tclfname = "../../data/vmd/patchyParticleSTMV_2vmd.tcl")
print("Percentage complete: ", 100, " %")

To load the movie go to /data/vmd and run in a terminal "vmd -e patchyPacrticles_2vmd.tcl".

## In case VMD is not desired

In case VMD output is not desired, below we simply integrate the model and show the output directly.

In [ ]:
# Integrate particle list and print only positions of first and last particle in list
timesteps = 10
print('{:<10s}{:<15s}{:<40s}{:<40s}'.format("Iteration", "Time", "Position (first particle)", "Position (last particle)"))
for i in range(timesteps):
    print('{:<10d}{:<15f}{:<40s}{:<50s}'.format(i, integrator.clock, str(partlist[0].position), str(partlist[numparticles-1].position)))
    print(np.linalg.norm(partlist[0].position - partlist[1].position))
    integrator.integrate(partlist)